<a href="https://colab.research.google.com/github/dcastf01/aena_scraping_licitaciones/blob/main/Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/dcastf01/aena_scrapping_licitaciones.git

Cloning into 'aena_scrapping_licitaciones'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 135 (delta 72), reused 50 (delta 25), pack-reused 0
Receiving objects: 100% (135/135), 7.69 MiB | 7.05 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [2]:
%cd aena_scrapping_licitaciones

/content/aena_scrapping_licitaciones


In [3]:
#@title Imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


In [4]:
#@title setup
filename = "clean_data_without_outliers.parquet" #@param ["clean_data_with_outliers.parquet", "clean_data_without_outliers.parquet"]
year_min_to_visualize = 2014 #@param {type:"integer"}
year_max_to_visualize = 2022 #@param {type:"integer"}

assert year_max_to_visualize>=year_min_to_visualize, 'please xlim_max must be greater than xlim_min'
folder_with_data='data'

input_file=os.path.join(folder_with_data,filename)
df=pd.read_parquet(input_file)

df=df[(
    df['Fecha de Contrato'].dt.year>=year_min_to_visualize) 
    & (df['Fecha de Contrato'].dt.year<=year_max_to_visualize)]
print('shape',df.shape)
print('initial colums \n',df.columns)
print('dtypes \n',df.dtypes)

shape (4898, 16)
initial colums 
 Index(['Fecha de Contrato', 'No Expediente', 'Destino', 'Titulo:',
       'Fecha de inicio', 'Fecha fin actual', 'Importe bruto licitacion:',
       'Importe bruto adjudicacion:', 'Forma adjudicacion:',
       'Numero de licitadores:', 'NIF', 'Razon social:', 'Fecha aprobacion',
       'duration_days', 'Importe bruto adjudicacion per day',
       '% bruto respect licitacion'],
      dtype='object')
dtypes 
 Fecha de Contrato                     datetime64[ns]
No Expediente                                 object
Destino                                       object
Titulo:                                       object
Fecha de inicio                       datetime64[ns]
Fecha fin actual                      datetime64[ns]
Importe bruto licitacion:                    float64
Importe bruto adjudicacion:                  float64
Forma adjudicacion:                           object
Numero de licitadores:                       float64
NIF                      

##Insights

### Creating report to have insights

###Main insights from this report
    * We have a Nº Expediente with several repetitions
    * We have a few values in the fields with "Importe" that break all the numeric análisis
    * Elecnor is the company with more "Licitaciones"
    * Exist contracts with 0
    * Contracts where the % bruto respect licitacion has the highest value or lowest value

##main insights from:
https://www.hacienda.gob.es/RSC/OIReScon/informe-anual-supervision-2020/ias-2020.pdf#page=30&zoom=100,101,132

* Cantidad de licitaciones por año
* Duración de licitaciones por año
* Importe bruto por día por año
* % bruto respecto al inicial
* Importe por año
Esas son las principales, luego cambiar la palabra año por aeropuerto / numero de licitaciones / la forma de adjudicación
* Numero de solicitudes de empresa por año
* Número de licitaciones por aeropuerto
* Importe por aeropuerto/departamento

* Company with more contracts




Licitaciones por año

In [142]:
import plotly.express as px
from pandas.api.types import is_datetime64_any_dtype as is_datetime
color_discrete_sequence= px.colors.sequential.Greens
def generate_grouper(df,column_to_group):
    grouper_column=[]
    for column in column_to_group:
        if  column is None or column not in df.columns:
            pass
        elif is_datetime(df[column]):
            grouper_column.append(df[column].dt.year)
        else:
            grouper_column.append(df[column])
    return grouper_column

def plot_bar_column(df,axis_y,axis_x=None,
                        color=None,variable_to_sort='column_axis_y',
                    function_to_group='sum',
                        is_ascending_sort_by_color=False,text_auto='.2s',**kwargs):
    if axis_x==color:
        grouper_column=generate_grouper(df,[axis_x])
    else:
        grouper_column=generate_grouper(df,[axis_x,color])
    # print(grouper_column)
    if function_to_group=='sum':
        df_to_plot=df.groupby(by=[*grouper_column])[axis_y].sum().reset_index()
    elif function_to_group=='count':
        df_to_plot=df.groupby(by=[*grouper_column])[axis_x].count().rename('count',axis=1).reset_index()
        axis_y='count'
    if isinstance( df_to_plot,pd.core.series.Series):
        df_to_plot=df_to_plot.to_frame()
    #solucionar lo de la parte del sort mientras se hace plot ya que no se me ordena como quiero, quizá se hace en el plot
    if variable_to_sort=='column_axis_y':
        column_to_sort=axis_y
    elif variable_to_sort=='choosing_dimension_to_colorbar':
        column_to_sort=color
  
    if color:
        df_to_plot.sort_values(axis_y,inplace=True,ascending=is_ascending_sort_by_color)
    print(df_to_plot.head())
    # return df_to_plot
    fig= px.bar(df_to_plot,x=axis_x,y=axis_y,color=color,text_auto='.2s',color_continuous_scale=color_discrete_sequence)
    return fig

In [143]:
#@title variables to print
# kind_of_plot = "bar_count" #@param ["bar_count","bar_sum"]
axis_x= "Numero de licitadores:" #@param ["Fecha de Contrato", "Destino","Forma adjudicacion:","Numero de licitadores:"]
axis_y= "Count" #@param ['duration_days','Importe bruto licitacion:','Count']
choosing_dimension_to_colorbar = "Numero de licitadores:" #@param ["None", "Destino", "Numero de licitadores:","Forma adjudicacion"]
sort_by_column_to_sum_or_by_dimension_to_colorbar="choosing_dimension_to_colorbar" #@param ['column_axis_y','choosing_dimension_to_colorbar']
is_ascending_sort_by_color = True #@param {type:"boolean"}


if choosing_dimension_to_colorbar=='None':
    choosing_dimension_to_colorbar=None

if axis_y =='Count':
    function_to_group='count'
else:
    function_to_group='sum'
    


plot_bar_column(df,
     axis_y,
     axis_x=axis_x,
     color=choosing_dimension_to_colorbar,
     is_ascending_sort_by_color=is_ascending_sort_by_color,
     function_to_group=function_to_group
                    ).show()


   Numero de licitadores:  count
7                     8.0    103
6                     7.0    104
5                     6.0    192
4                     5.0    275
3                     4.0    561


In [131]:
df.head()

,Fecha de Contrato,No Expediente,Destino,Titulo:,Fecha de inicio,Fecha fin actual,Importe bruto licitacion:,Importe bruto adjudicacion:,Forma adjudicacion:,Numero de licitadores:,NIF,Razon social:,Fecha aprobacion,duration_days,Importe bruto adjudicacion per day,% bruto respect licitacion
2,2022-07-21,IBZ-32/2022,APTO.IBIZA,MTTO. PREVENT. CORREC. ASIST. TELEF.CONSULTAS ...,2022-08-21,2023-08-21,12402.50,12402.50,PROCEDIMIENTO ABIERTO SIMPLIFICADO,1.0,A058090713,IBERSELEX SA,0.0,365.0,33.979452,1.000000
25,2022-07-08,PMI-33/2022,APTO.PALMA DE MALLORCA,NUEVA BASCULA DE PESAJE PARA LA PLANTA DE TRAN...,2022-07-14,2022-12-14,192798.86,191834.86,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,1.0,A007296536,AMER E HIJOS SA,0.0,153.0,1253.822614,0.995000
27,2022-07-08,PMI-10/2022,APTO.PALMA DE MALLORCA,"SERVICIO DE REVISIÓN, ACTUALIZACIÓN Y MANTENIM...",2022-07-25,2023-07-25,133100.00,81130.50,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,4.0,B065322869,ALDEBARAN 2 0 IT PROJ SL,0.0,365.0,222.275342,0.609545
31,2022-07-07,MAD-85/2022,APTO.A.SUAREZ MAD-BARAJAS,ALQUILER DE EQUIPOS DE DESINFECCIÓN DE BANDEJA...,2022-07-20,2023-07-26,87120.00,87120.00,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,1.0,B030860332,"TECNIMUSA, S.L.",0.0,371.0,234.824798,1.000000
79,2022-06-23,MAD-45/2022,APTO.A.SUAREZ MAD-BARAJAS,SUMINISTRO E INSTALACIÓN DE EQUIPAMIENTO HARDW...,2022-07-11,2023-04-11,220540.65,161958.50,PROCEDIMIENTO ABIERTO VARIOS CRITERIOS,2.0,A079486833,ELECNOR SERV PROYECTOS SA,0.0,274.0,591.089416,0.734370


### contracts with import 0

In [ ]:
df[df['Importe bruto adjudicación(€)']==0]

###Checking what happen with the "N* Expediente" most repeated

In [ ]:
df[df['Nº Expediente']=='DTC-83/2015-1']

### We have a value anomalous in relacion "importes" 


In [ ]:
df.sort_values('Importe neto adjudicación(€)',ascending=False) 

Before to check the outliers for us is interest know what are the contract where % bruto respect licitación was highest or lowest. We only are interested in know the date where and to who

In [ ]:
display('Highest',
      df[['Fecha de Contrato','Destino','Razón social:','% bruto respect licitacion']
         ].sort_values('% bruto respect licitacion',ascending=False).head(10)
      )
display('Lowest',
      df[['Fecha de Contrato','Destino','Razón social:','% bruto respect licitacion']
         ].sort_values('% bruto respect licitacion').head(10)
      )

###create report without outlier

In [ ]:
print('shape before remove outlier', df.shape)
df_wihtout_outliers=df[df['Importe bruto adjudicación(€)']<=1e+06]
print('shape after remove outlier', df_wihtout_outliers.shape)

In [ ]:
report=create_report(df_wihtout_outliers.drop(['Importe bruto licitación:','duration_days'],axis=1))
report.save('without_outliers.html')